# Search for AIMA 4th edition

Implementation of search algorithms and search problems for AIMA.

# Problems and Nodes

We start by defining the abstract class for a `Problem`; specific problem domains will subclass this. To make it easier for algorithms that use a heuristic evaluation function, `Problem` has a default `h` function (uniformly zero), and subclasses can define their own default `h` function.

We also define a `Node` in a search tree, and some functions on nodes: `expand` to generate successors; `path_actions` and `path_states`  to recover aspects of the path from the node.  

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random
import heapq
import math
import sys, os
import time
import pandas as pd
import os

sys.path.append(os.getcwd())
#import bisect
from bisect import *

from collections import defaultdict, deque, Counter
from itertools import combinations


class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When you create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)
    

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

# Queues

First-in-first-out and Last-in-first-out queues, and a `PriorityQueue`, which allows you to keep a collection of items, and continually remove from it the item with minimum `f(item)` score.

In [2]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

# Search Algorithms: Best-First

Best-first search with various *f(n)* functions gives us different search algorithms. Note that A\*, weighted A\* and greedy search can be given a heuristic function, `h`, but if `h` is not supplied they use the problem's default `h` function (if the problem does not define one, it is taken as *h(n)* = 0).

In [3]:
# Save results to excel file by passing the name of the sheet and the data
# This will append the new date to a new sheet in the excel file
def save_to_excel(t, NODES_EXPANDED, len_reached, len_frontier, path_cost, len_node, STATES_REEXPANDED, DUMMY_NODES, sheet):
    output_file_path = "results_heuristics.xlsx"
    df = pd.DataFrame([[t, NODES_EXPANDED, (t/NODES_EXPANDED), len_reached, len_frontier, path_cost, len_node, STATES_REEXPANDED, DUMMY_NODES]], columns=['Time', 'Nodes Expanded', 'Time per node', 'States Reached', 'Nodes in Frontier', 'Solution Cost', 'Solution Length', 'Reexpanded States', 'Dummy Nodes'])
    # Check if the file exists
    if not os.path.exists(output_file_path):
        pd.ExcelWriter(output_file_path, engine='openpyxl',mode='a', if_sheet_exists='overlay')
        # If the file does not exist, create it and save the DataFrame to it
        df.to_excel(output_file_path, sheet_name=sheet, index=False)
        print(f"File '{output_file_path}' created and data saved.")
    else:
        # If the file exists, append the DataFrame to the existing file
        data1 = pd.read_excel(output_file_path, sheet_name=sheet)
        df = pd.concat([df, data1], axis=0, ignore_index=True)
        with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            df.to_excel(writer, index=False, sheet_name=sheet)
        print(f"Data appended to '{output_file_path}'.")


def best_first_search(problem, sheet, f):
    # Search nodes with minimum f(node) value first.
    seconds = time.time()
    NODES_EXPANDED = 0
    STATES_REEXPANDED = 0
    DUMMY_NODES = 0
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    expanded = {}
    while frontier:
        # node extraction, no  dummy nodes control 
        node = frontier.pop()
        
        # Dummy: nodes where there's a better solution for that same state
        while(node.path_cost > reached[node.state].path_cost):
            DUMMY_NODES += 1
            node = frontier.pop()

        # Reexpansion control, number of states reexpanded
        if(node.state in expanded):
            STATES_REEXPANDED += 1
        else:
            expanded[node.state] = True
        
        # counting expansions
        NODES_EXPANDED += 1
        
        # Monitoring expanded states and f-values
        # print(node.state, "f()= ", f(node))
     
        #if goal, show statistics
        if problem.is_goal(node.state):
            t = time.time()-seconds
            print("Time Taken: ", t)
            print("Nodes Expanded: ", NODES_EXPANDED)
            print("Time per node: ", t/NODES_EXPANDED)
            print("States Reached: ", len(reached))
            print("Nodes in Frontier: ", len(frontier))
            print("Solution Cost: ", node.path_cost)
            print("Solution Length: ", len(node))
            print("Reexpanded States: ", STATES_REEXPANDED)
            print("Discarded Dummy Nodes: ", DUMMY_NODES)

            save_to_excel(t, NODES_EXPANDED, len(reached), len(frontier), node.path_cost, len(node), STATES_REEXPANDED, DUMMY_NODES, sheet)

            return node

        # node expansion
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, sheet, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, sheet, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)



# Other Search Algorithms

Here are the other search algorithms:

In [4]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
        
        
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


def depth_first_recursive_search(problem, node=None):
    if node is None: 
        node = Node(problem.initial)
    if problem.is_goal(node.state):
        return node
    elif is_cycle(node):
        return failure
    else:
        for child in expand(problem, node):
            result = depth_first_recursive_search(problem, child)
            if result != failure:
                return result
        return failure

In [5]:
#path_states(depth_first_recursive_search(r2))

# Bidirectional Best-First Search

In [6]:
def bidirectional_best_first_search(problem_f, f_f, problem_b, f_b, terminated):
    node_f = Node(problem_f.initial)
    node_b = Node(problem_f.goal)
    frontier_f, reached_f = PriorityQueue([node_f], key=f_f), {node_f.state: node_f}
    frontier_b, reached_b = PriorityQueue([node_b], key=f_b), {node_b.state: node_b}
    solution = failure
    while frontier_f and frontier_b and not terminated(solution, frontier_f, frontier_b):
        def S1(node, f):
            return str(int(f(node))) + ' ' + str(path_states(node))
        print('Bi:', S1(frontier_f.top(), f_f), S1(frontier_b.top(), f_b))
        if f_f(frontier_f.top()) < f_b(frontier_b.top()):
            solution = proceed('f', problem_f, frontier_f, reached_f, reached_b, solution)
        else:
            solution = proceed('b', problem_b, frontier_b, reached_b, reached_f, solution)
    return solution

def inverse_problem(problem):
    if isinstance(problem, CountCalls):
        return CountCalls(inverse_problem(problem._object))
    else:
        inv = copy.copy(problem)
        inv.initial, inv.goal = inv.goal, inv.initial
        return inv

In [7]:
def bidirectional_uniform_cost_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        n_f, n_b = frontier_f.top(), frontier_b.top()
        return g(n_f) + g(n_b) > g(solution)
    return bidirectional_best_first_search(problem_f, g, inverse_problem(problem_f), g, terminated)

def bidirectional_astar_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        nf, nb = frontier_f.top(), frontier_b.top()
        return g(nf) + g(nb) > g(solution)
    problem_f = inverse_problem(problem_f)
    return bidirectional_best_first_search(problem_f, lambda n: g(n) + problem_f.h(n),
                                           problem_b, lambda n: g(n) + problem_b.h(n), 
                                           terminated)
   

def proceed(direction, problem, frontier, reached, reached2, solution):
    node = frontier.pop()
    for child in expand(problem, node):
        s = child.state
        print('proceed', direction, S(child))
        if s not in reached or child.path_cost < reached[s].path_cost:
            frontier.add(child)
            reached[s] = child
            if s in reached2: # Frontiers collide; solution found
                solution2 = (join_nodes(child, reached2[s]) if direction == 'f' else
                             join_nodes(reached2[s], child))
                #print('solution', path_states(solution2), solution2.path_cost, 
                # path_states(child), path_states(reached2[s]))
                if solution2.path_cost < solution.path_cost:
                    solution = solution2
    return solution

S = path_states

#A-S-R + B-P-R => A-S-R-P + B-P
def join_nodes(nf, nb):
    """Join the reverse of the backward node nb to the forward node nf."""
    #print('join', S(nf), S(nb))
    join = nf
    while nb.parent is not None:
        cost = join.path_cost + nb.path_cost - nb.parent.path_cost
        join = Node(nb.parent.state, join, nb.action, cost)
        nb = nb.parent
        #print('  now join', S(join), 'with nb', S(nb), 'parent', S(nb.parent))
    return join
    
   

In [8]:
#A , B = uniform_cost_search(r1), uniform_cost_search(r2)
#path_states(A), path_states(B)

In [9]:
#path_states(append_nodes(A, B))

# TODO: RBFS

# Problem Domains

Now we turn our attention to defining some problem domains as subclasses of `Problem`.

# Symmetric TSP

![](romania.png)

En el TSP simetrico, los estados son pares de la forma [CiudadesVisitadas, CiudadActual]. El estado inicial es [ConjuntoVacio, [A]], el estado objetivo es de la forma [TodasLasCiudades, [A]]. Los sucesores de un estado se generan a partir de las ciudades no visitadas, uno por cada una. El coste de una regla es el coste entre la ciudad actual y la nueva ciudad visitada que pasa a ser la actual

In [10]:
class EstadoTSP: #un estado es un subproblema sin resolver, no necesita el orden en que se han recorrido las ciudades ya visitadas
    def __init__(self, actual, visitadas):
        self.actual = actual
        self.visitadas = visitadas
        lista = visitadas.copy()
        lista.append(actual)
        self.state = tuple(lista)
       # self.hash = hash(self.state)
        self.str = " [Visitadas: " + str(self.visitadas) + " Actual: " + str(self.actual) +"]"
    def __hash__(self):
        return hash(self.state)
        #return self.hash
    def __eq__(self,other):
        return self.actual == other.actual and self.visitadas == other.visitadas
    def __str__(self):
        return " [Visitadas: " + str(self.visitadas) + " Actual: " + str(self.actual) +"]"
    def __repr__(self):
        return " [Visitadas: " + str(self.visitadas) + " Actual: " + str(self.actual) +"]"


In [11]:
class Grafo:
    """Un Grafo es un array bidimensional de tamño N*N, siendo N el número de ciudades,
    representadas por los valores 0,...,N-1; 0 se toma como ciudad de partida.
    El valor de la posición (i,j) es la distancia entre las ciudades i y j,
     que es el mismo que la distancia entre j e i """
    def __init__(self,lista):
        "N es el número de ciudades correspondiente a la lista de valores de una instancia en la sintaxis EDGE_WEIGHT_TYPE"
        self.N = int(((8*len(lista)+1)**0.5)-1)/2
        # print(self.N)
        self.ciudades = list(range(int(self.N)))
        self.minarcs = [math.inf] * (int(self.N))
        self.Dist = [[0]*int(self.N) for i in self.ciudades]
        x_acc = 0
        for x in range(int(self.N)):
            x_acc += x
            for y in range(x+1):
                self.Dist[x][y] = self.Dist[y][x] = lista[x_acc+y]

                if lista[x_acc+y] != 0 and lista[x_acc+y] < self.minarcs[x]:
                    self.minarcs[x] = lista[x_acc+y]

                if lista[x_acc+y] != 0 and lista[x_acc+y] < self.minarcs[y]:
                    self.minarcs[y] = lista[x_acc+y]
    
        



In [12]:
class KruskalGraph:
    def __init__(self, vertices):  
        self.vertices = vertices  
        self.edges = []  
        self.adjacency_list = {v: [] for v in vertices}  
  
    def add_edge(self, u, v, weight):  
        self.edges.append((u, v, weight))  
        self.adjacency_list[u].append((v, weight))  
        self.adjacency_list[v].append((u, weight))  
  
    def find_parent(self, parent, i):  
        if parent[i] == i:  
            return i  
        return self.find_parent(parent, parent[i])  
  
    def union(self, parent, rank, x, y):  
        root_x = self.find_parent(parent, x)  
        root_y = self.find_parent(parent, y)  
        if rank[root_x] < rank[root_y]:  
            parent[root_x] = root_y  
        elif rank[root_x] > rank[root_y]:  
            parent[root_y] = root_x  
        else:  
            parent[root_y] = root_x  
            rank[root_x] += 1  
  
    def kruskal(self):  
        minimum_spanning_tree = set()  
        parent = {}  
        rank = {}  
        for v in self.vertices:  
            parent[v] = v  
            rank[v] = 0  
        sorted_edges = sorted(self.edges, key=lambda x: x[2])  
        for edge in sorted_edges:  
            u, v, weight = edge  
            root_u = self.find_parent(parent, u)  
            root_v = self.find_parent(parent, v)  
            if root_u != root_v:  
                minimum_spanning_tree.add(edge)  
                self.union(parent, rank, root_u, root_v)  
        return minimum_spanning_tree  

In [13]:
class SymmetricTSP(Problem):
    """El problema consiste en encontrar un camino hamiltoniano en un grafo no dirigido 
    y totalmente conectado, con costes positivos en los ejes. Las ciudades son 0,...,N-1
    La ciudad de partida es la 0"""

    def __init__(self, grafo):
        self.grafo = grafo
        self.initial = EstadoTSP(0,[])
        #print(self.__dir__initial)

    def actions(self, state):
        """Una accion por cada ciudad no visitada distinta de la actual, si todas visitadas -1"""
        noVisitadas = []
        for c in self.grafo.ciudades: 
            if c not in state.visitadas and c != state.actual:
                noVisitadas.append(c) 
        if len(noVisitadas)==0:
            noVisitadas = [-1]
        return noVisitadas
    
    def result(self, state, action): 
        setVisitadas = list(state.visitadas)
        insort(setVisitadas,state.actual)
        return EstadoTSP(action,setVisitadas)
    
    def is_goal(self, state):        return state.actual == -1

    def action_cost(self, s, a, s1): 
        if (len(s.visitadas)==self.grafo.N-1):
            return self.grafo.Dist[s.actual][0]
        else:
            return self.grafo.Dist[s.actual][a]
        
    # TODO HEURISTICOS, LO MAS IMPORTANTE!!
   
    def h_POBRE(self,node):
        # heuristico muy poco informado, cuenta el numero de ciudades que faltan por visitar, mas o menos
        return self.grafo.N - len(node.state.visitadas) 

    def h1(self, node):
        # Heuristico que considera los arcos minimos de las ciudades
        # que quedan por abandonar. 
        # No responde a ninguna relajacion, aparentemente
        result = 0
        for ciudad in self.grafo.ciudades:
            if ciudad not in node.state.visitadas:
                arcos = self.obtener_arcos_no_visitados(ciudad, node)
                if arcos:
                    min_arco = min(arcos)
                    result += min_arco
        return result

    def obtener_arcos_no_visitados(self, ciudad, node):
        arcos = []
        for idx, coste in enumerate(self.grafo.Dist[ciudad]):
            if coste > 0 and idx not in node.state.visitadas:
                arcos.append(coste)
        return arcos
    
    def grafoResidual(self,node):
        #DONE dado un Grafo, crear el grafo residual:
        # todos los arcos a las ciudades no visitadas, más el arco de la actual a la objetivo
        # 1) crear lista de valores
        # 2) nueva matriz sólo con las filas y columnas no visitadas
        # 3) arco de la actual a la inicial con valor infinito
        
        # 1)
        noVisitadas = self.actions(node.state)
        if(noVisitadas == -1):
            return 0
        
        noVisitadas.append(node.state.actual)
        noVisitadas.sort()
        
        #2)
        grafoResidual = [[0]*len(noVisitadas) for i in noVisitadas]
        
        for i in range(len(noVisitadas)):
            for j in range(len(noVisitadas)):
                grafoResidual[i][j] = self.grafo.Dist[noVisitadas[i]][noVisitadas[j]]
        
        #3)
        ciudadActual = noVisitadas.index(node.state.actual)
        #grafoResidual[0][ciudadActual] = grafoResidual[ciudadActual][0] = math.inf
        
        return grafoResidual
    
    def h2(self,node):
        # Relajacion R2, R3, R4.
        # Suma de los N-k+1 arcos de menos coste del grafo residual

        noVisitadas = self.actions(node.state)
        if(noVisitadas == [-1]):
            return 0
        
        noVisitadas.append(node.state.actual)
        noVisitadas.sort()
        
        listaResidual = [0]*sum(range(1,len(noVisitadas)+1))
        
        x_acc = 0
        for x in range(len(noVisitadas)):
            x_acc += x
            for y in range(x+1):
                listaResidual[x_acc+y] = self.grafo.Dist[noVisitadas[y]][noVisitadas[x]]
        
        noZeros = [i for i in listaResidual if i != 0]
        noZeros.sort()
        a = int(self.grafo.N)-len(node.state.visitadas)-1
        b = noZeros[:a]
        return sum(b)
    
    def h3(self,node):
        # Relajacion R3, R4
        # Suma de los N-k+1 arcos de menos coste del grafo residual
        # tales que para cada ciudad del grafo residual hay un arco que la toca
        # ¿Que algoritmo calcula esto? Hay que inventarlo
        
        noVisitadas = self.actions(node.state)
        if(noVisitadas == -1):
            return 0
        
        noVisitadas.append(node.state.actual)
        noVisitadas.sort()
        
        # Calcular grafo residual
        grafoRes = np.matrix(self.grafoResidual(node)).astype(float)
        
        grafoRes[grafoRes == 0] = math.inf
        
        acc = 0
        for ciudad in range(len(noVisitadas)-1):
            # minimum arc in subgaph
            ij_min = np.unravel_index(grafoRes.argmin(), grafoRes.shape)
            minVal = grafoRes[ij_min]
            if minVal == math.inf:
                continue
            
            acc += minVal
            
            # remove row and column of that city (can no longer be "reached")
            grafoRes = np.delete(grafoRes, ij_min[0], 0)
            grafoRes = np.delete(grafoRes, ij_min[1], 1)
        
        # Hacer análisis experimental de si es admisible, consistente, monótono... decir si puede que lo sea o no
        return acc
    
    
    def h_MST(self, node):
        # Relajacion R3
        # Coste de un arbol de expansion minimo del grafo residual
        noVisitadas = self.actions(node.state)
        if(noVisitadas == -1):
            return 0
        
        noVisitadas.append(node.state.actual)
        noVisitadas.sort()
        
        g = KruskalGraph(noVisitadas)
        
        for ciudadSalida in noVisitadas:
            for ciudadEntrada in noVisitadas:
                g.add_edge(ciudadSalida, ciudadEntrada, self.grafo.Dist[ciudadSalida][ciudadEntrada])
        
        result = g.kruskal()
        
        return sum(w for u, v, w in result)
    
    # Current heuristic
    def h(self,node):
        return self.h_MST(node)


In [14]:
# Datos de instancias en formato EDGE_WEIGHT_TYPE de la TSPLIB

ejemploClase = [ 
    0,
    21, 0,
    20, 32, 0,
    130, 302, 108, 0,
    80, 25, 18, 25, 0,
    32, 9, 50, 39, 27, 0
]

gr24 = [0,257,0,187,196,0,91,228,158,0,150,112
,96,120,0,80,196,88,77,63,0,130,167,59
,101,56,25,0,134,154,63,105,34,29,22,0
,243,209,286,159,190,216,229,225,0,185,86,124
,156,40,124,95,82,207,0,214,223,49,185,123
,115,86,90,313,151,0,70,191,121,27,83,47
,64,68,173,119,148,0,272,180,315,188,193,245
,258,228,29,159,342,209,0,219,83,172,149,79
,139,134,112,126,62,199,153,97,0,293,50,232
,264,148,232,203,190,248,122,259,227,219,134,0
,54,219,92,82,119,31,43,58,238,147,84,53
,267,170,255,0,211,74,81,182,105,150,121,108
,310,37,160,145,196,99,125,173,0,290,139,98
,261,144,176,164,136,389,116,147,224,275,178,154
,190,79,0,268,53,138,239,123,207,178,165,367
,86,187,202,227,130,68,230,57,86,0,261,43
,200,232,98,200,171,131,166,90,227,195,137,69
,82,223,90,176,90,0,175,128,76,146,32,76
,47,30,222,56,103,109,225,104,164,99,57,112
,114,134,0,250,99,89,221,105,189,160,147,349
,76,138,184,235,138,114,212,39,40,46,136,96
,0,192,228,235,108,119,165,178,154,71,136,262
,110,74,96,264,187,182,261,239,165,151,221,0
,121,142,99,84,35,29,42,36,220,70,126,55
,249,104,178,60,96,175,153,146,47,135,169,0]
gr48= [0,593,0,409,258,0,566,331,171,0
,633,586,723,874,0,257,602,522,679,390
,0,91,509,325,482,598,228,0,412,627
,506,663,227,169,383,0,378,755,634,791
,397,175,349,167,0,593,416,564,721,271
,445,509,293,429,0,150,598,414,571,488
,112,120,267,233,541,0,659,488,630,787
,205,511,575,304,470,76,607,0,80,566
,382,539,572,196,77,351,317,563,63,629
,0,434,893,699,856,524,231,405,303,138
,595,289,606,373,0,455,417,433,590,313
,304,371,228,394,158,399,224,425,530,0
,134,583,399,566,530,154,105,309,275,575
,34,638,29,298,434,0,649,945,824,981
,446,423,620,357,280,649,504,648,588,416
,584,546,0,259,364,180,337,555,272,175
,338,466,403,264,469,232,549,265,249,656
,0,505,354,110,70,819,618,421,602,730
,660,509,728,478,795,529,494,920,276,0
,710,117,375,354,679,693,626,720,848,533
,715,610,683,986,534,700,1038,481,345,0
,488,784,663,820,289,262,459,196,119,488
,343,502,427,255,423,385,161,495,759,877
,0,353,641,520,677,282,110,324,61,125
,353,208,364,292,261,288,250,315,352,616
,734,154,0,324,275,91,248,638,437,240
,421,549,486,329,552,297,614,348,314,739
,95,187,392,578,435,0,605,287,431,588
,313,445,520,470,598,143,610,215,577,734
,144,595,788,352,527,404,627,484,385,0
,372,229,39,196,686,485,288,469,597,511
,397,578,345,662,396,361,787,143,135,346
,626,483,54,377,0,330,484,361,518,378
,119,260,150,278,323,174,389,276,414,185
,207,468,193,475,577,307,164,276,326,324
,0,581,877,756,913,370,355,552,289,212
,581,436,571,520,348,516,478,84,588,852
,970,93,247,671,720,719,400,0,154,460
,276,433,612,298,63,453,419,460,190,526
,158,475,322,175,690,126,372,577,529,396
,191,471,239,250,622,0,70,523,339,496
,569,191,27,346,312,516,83,589,47,368
,385,68,583,189,435,640,422,287,254,534
,302,249,515,115,0,606,183,216,147,715
,719,522,703,831,549,611,615,579,896,546
,596,1021,377,139,209,860,717,288,416,242
,558,953,473,536,0,585,427,563,720,179
,437,501,196,362,80,532,108,558,498,163
,567,552,395,659,544,391,256,478,154,526
,318,484,452,515,556,0,544,840,719,876
,311,318,515,252,175,508,399,494,483,311
,479,441,154,551,815,933,65,210,634,683
,682,363,77,585,479,916,399,0,496,525
,595,751,147,253,468,85,251,208,351,236
,435,387,162,393,441,427,691,646,280,145
,509,249,558,239,373,538,430,654,128,336
,0,317,289,105,262,631,430,233,414,542
,479,332,545,290,607,341,307,732,88,201
,406,571,428,21,407,68,269,664,184,247
,302,471,627,503,0,648,68,316,362,584
,598,564,625,753,418,653,484,621,891,415
,638,943,395,412,95,782,639,333,285,287
,482,875,515,578,209,425,838,523,347,0
,211,660,476,633,466,74,182,243,171,489
,66,555,150,227,351,108,432,326,572,777
,271,184,391,492,439,166,364,252,145,673
,438,327,327,384,715,0,475,137,295,452
,437,428,391,452,580,271,480,337,448,718
,268,465,770,222,391,254,609,466,255,138
,241,309,702,342,405,287,278,665,376,277
,167,542,0,654,151,319,266,755,767,570
,751,879,561,659,627,627,944,558,644,1069
,425,262,103,908,765,336,428,290,606,1001
,521,584,122,568,964,666,350,169,721,299
,0,710,239,487,546,616,660,626,687,815
,443,715,509,683,953,440,700,1005,457,583
,279,844,701,490,310,458,544,937,577,640
,393,450,900,548,512,179,777,229,353,0
,585,135,385,458,499,535,501,562,690,333
,590,399,558,828,330,575,880,332,481,215
,719,576,365,200,356,419,812,452,515,318
,340,775,438,387,120,652,104,289,121,0
,246,373,183,340,745,472,237,528,656,593
,364,659,332,649,455,349,846,202,279,490
,685,542,157,525,144,383,778,174,289,386
,585,741,618,132,431,426,395,434,630,505
,0,788,208,456,488,724,738,704,765,893
,558,793,624,761,1031,555,778,1083,535,552
,188,922,779,473,425,427,622,1015,655,718
,343,565,978,663,487,138,855,307,284,138
,235,571,0,446,162,111,268,624,559,362
,543,671,458,451,524,419,736,455,436,861
,217,207,279,700,557,128,325,82,398,793
,313,376,175,465,756,563,142,220,513,187
,223,391,289,226,360,0,166,437,247,404
,749,435,150,590,556,597,402,663,295,612
,459,387,827,189,343,554,666,531,221,589
,208,372,759,137,177,450,589,722,675,196
,495,389,459,498,694,569,80,635,290,0
,523,81,188,255,596,636,439,620,648,430
,528,496,496,813,427,513,938,294,284,193
,777,634,205,297,159,475,870,390,453,119
,437,833,535,219,139,590,168,131,310,208
,303,279,92,367,0,235,371,187,344,581
,348,151,364,469,429,240,495,208,525,291
,225,682,32,283,488,521,378,103,384,150
,219,614,94,165,384,421,577,454,92,429
,302,254,432,489,364,165,569,224,154,301
,0,369,205,289,446,537,328,286,355,483
,371,375,437,343,554,269,360,673,116,385
,322,512,369,149,238,230,209,605,237,300
,352,378,568,445,172,281,436,108,332,343
,218,290,421,164,354,201,149,0,121,570
,386,543,518,142,84,297,263,570,35,636
,29,319,432,36,534,236,482,687,373,238
,301,581,349,222,466,162,55,583,562,429
,381,294,625,96,452,631,687,562,336,765
,423,299,500,212,347,0]

gr21 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,
]

gr17 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 
    246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 
    121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0,
 ]

instance = SymmetricTSP(Grafo(gr17))
instance2 = SymmetricTSP(Grafo(gr21))
instance3 = SymmetricTSP(Grafo(gr24))
instance4 = SymmetricTSP(Grafo(gr48))
#path_actions(astar_search(instance))
numrepeat = 20
for i in range(numrepeat):
    path_states(astar_search(instance, "h1", instance.h1))
for i in range(numrepeat):
    path_states(astar_search(instance, "h2", instance.h2))
for i in range(numrepeat):
    path_states(astar_search(instance, "h3", instance.h3))
for i in range(numrepeat):
    path_states(astar_search(instance, "h_MST", instance.h_MST))
for i in range(numrepeat):
    path_states(astar_search(instance2, "h1", instance2.h1))
for i in range(numrepeat):
    path_states(astar_search(instance2, "h2", instance2.h2))
for i in range(numrepeat):
    path_states(astar_search(instance2, "h3", instance2.h3))
for i in range(numrepeat):
    path_states(astar_search(instance2, "h_MST", instance2.h_MST))
for i in range(numrepeat):
    path_states(astar_search(instance3, "h1", instance3.h1))
for i in range(numrepeat):
    path_states(astar_search(instance3, "h2", instance3.h2))
for i in range(numrepeat):
    path_states(astar_search(instance3, "h3", instance3.h3))
for i in range(numrepeat):
    path_states(astar_search(instance3, "h_MST", instance3.h_MST))
for i in range(numrepeat):
    path_states(astar_search(instance4, "h1", instance4.h1))
for i in range(numrepeat):
    path_states(astar_search(instance4, "h2", instance4.h2))
for i in range(numrepeat):
    path_states(astar_search(instance4, "h3", instance4.h3))
for i in range(numrepeat):
    path_states(astar_search(instance4, "h_MST", instance4.h_MST))



Time Taken:  10.620328903198242
Nodes Expanded:  94232
Time per node:  0.00011270405916459634
States Reached:  257193
Nodes in Frontier:  232631
Solution Cost:  2085
Solution Length:  17
Reexpanded States:  2
Discarded Dummy Nodes:  31000
Data appended to 'results_heuristics.xlsx'.
Time Taken:  10.582086563110352
Nodes Expanded:  94232
Time per node:  0.00011229822738677255
States Reached:  257193
Nodes in Frontier:  232631
Solution Cost:  2085
Solution Length:  17
Reexpanded States:  2
Discarded Dummy Nodes:  31000
Data appended to 'results_heuristics.xlsx'.
Time Taken:  10.701212882995605
Nodes Expanded:  94232
Time per node:  0.00011356240855543346
States Reached:  257193
Nodes in Frontier:  232631
Solution Cost:  2085
Solution Length:  17
Reexpanded States:  2
Discarded Dummy Nodes:  31000
Data appended to 'results_heuristics.xlsx'.
Time Taken:  10.604925155639648
Nodes Expanded:  94232
Time per node:  0.00011254059295822702
States Reached:  257193
Nodes in Frontier:  232631
Soluti

KeyboardInterrupt: 

In [ ]:
instance = SymmetricTSP(Grafo(gr17))
#path_actions(astar_search(instance))
path_states(astar_search(instance))

TypeError: astar_search() missing 1 required positional argument: 'sheet'

# Route Finding Problems

![](romania.png)

In a `RouteProblem`, the states are names of "cities" (or other locations), like `'A'` for Arad. The actions are also city names; `'Z'` is the action to move to city `'Z'`. The layout of cities is given by a separate data structure, a `Map`, which is a graph where there are vertexes (cities), links between vertexes, distances (costs) of those links (if not specified, the default is 1 for every link), and optionally the 2D (x, y) location of each city can be specified. A `RouteProblem` takes this `Map` as input and allows actions to move between linked cities. The default heuristic is straight-line distance to the goal, or is uniformly zero if locations were not given.

In [ ]:
class RouteProblem(Problem):
    """A problem to find a route between locations on a `Map`.
    Create a problem with RouteProblem(start, goal, map=Map(...)}).
    States are the vertexes in the Map graph; actions are destination states."""
    
    def actions(self, state): 
        """The places neighboring `state`."""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def action_cost(self, s, action, s1):
        """The distance (cost) to go from s to s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        return straight_line_distance(locs[node.state], locs[self.goal])
    
    
def straight_line_distance(A, B):
    "Straight-line distance between two points."
    return sum(abs(a - b)**2 for (a, b) in zip(A, B)) ** 0.5

In [ ]:
class Map:
    """A map of places in a 2D world: a graph with vertexes and links between them. 
    In `Map(links, locations)`, `links` can be either [(v1, v2)...] pairs, 
    or a {(v1, v2): distance...} dict. Optional `locations` can be {v1: (x, y)} 
    If `directed=False` then for every (v1, v2) link, we add a (v2, v1) link."""

    def __init__(self, links, locations=None, directed=False):
        if not hasattr(links, 'items'): # Distances are 1 by default
            links = {link: 1 for link in links}
        if not directed:
            for (v1, v2) in list(links):
                links[v2, v1] = links[v1, v2]
        self.distances = links
        self.neighbors = multimap(links)
        self.locations = locations or defaultdict(lambda: (0, 0))

        
def multimap(pairs) -> dict:
    "Given (key, val) pairs, make a dict of {key: [val,...]}."
    result = defaultdict(list)
    for key, val in pairs:
        result[key].append(val)
    return result

In [ ]:
# Some specific RouteProblems

romania = Map(
    {('O', 'Z'):  71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
     ('L', 'T'): 111, ('L', 'M'):  70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
     ('C', 'P'): 138, ('R', 'S'):  80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
     ('B', 'G'):  90, ('B', 'U'):  85, ('H', 'U'): 98, ('E', 'H'):  86, ('U', 'V'): 142, 
     ('I', 'V'):  92, ('I', 'N'):  87, ('P', 'R'): 97},
    {'A': ( 76, 497), 'B': (400, 327), 'C': (246, 285), 'D': (160, 296), 'E': (558, 294), 
     'F': (285, 460), 'G': (368, 257), 'H': (548, 355), 'I': (488, 535), 'L': (162, 379),
     'M': (160, 343), 'N': (407, 561), 'O': (117, 580), 'P': (311, 372), 'R': (227, 412),
     'S': (187, 463), 'T': ( 83, 414), 'U': (471, 363), 'V': (535, 473), 'Z': (92, 539)})


r0 = RouteProblem('A', 'A', map=romania)
r1 = RouteProblem('A', 'B', map=romania)
r2 = RouteProblem('N', 'L', map=romania)
r3 = RouteProblem('E', 'T', map=romania)
r4 = RouteProblem('O', 'M', map=romania)

In [ ]:
path_states(uniform_cost_search(r1)) # Lowest-cost path from Arab to Bucharest

In [ ]:
path_states(breadth_first_search(r1)) # Breadth-first: fewer steps, higher path cost

# Grid Problems

A `GridProblem` involves navigating on a 2D grid, with some cells being impassible obstacles. By default you can move to any of the eight neighboring cells that are not obstacles (but in a problem instance you can supply a `directions=` keyword to change that). Again, the default heuristic is straight-line distance to the goal. States are `(x, y)` cell locations, such as `(4, 2)`, and actions are `(dx, dy)` cell movements, such as `(0, -1)`, which means leave the `x` coordinate alone, and decrement the `y` coordinate by 1.

In [ ]:
class GridProblem(Problem):
    """Finding a path on a 2D grid with obstacles. Obstacles are (x, y) cells."""

    def __init__(self, initial=(15, 30), goal=(130, 30), obstacles=(), **kwds):
        Problem.__init__(self, initial=initial, goal=goal, 
                         obstacles=set(obstacles) - {initial, goal}, **kwds)

    directions = [(-1, -1), (0, -1), (1, -1),
                  (-1, 0),           (1,  0),
                  (-1, +1), (0, +1), (1, +1)]
    
    def action_cost(self, s, action, s1): return straight_line_distance(s, s1)
    
    def h(self, node): return straight_line_distance(node.state, self.goal)
                  
    def result(self, state, action): 
        "Both states and actions are represented by (x, y) pairs."
        return action if action not in self.obstacles else state
    
    def actions(self, state):
        """You can move one cell in any of `directions` to a non-obstacle cell."""
        x, y = state
        return {(x + dx, y + dy) for (dx, dy) in self.directions} - self.obstacles
    
class ErraticVacuum(Problem):
    def actions(self, state): 
        return ['suck', 'forward', 'backward']
    
    def results(self, state, action): return self.table[action][state]
    
    table = dict(suck=    {1:{5,7}, 2:{4,8}, 3:{7}, 4:{2,4}, 5:{1,5}, 6:{8}, 7:{3,7}, 8:{6,8}},
                 forward= {1:{2}, 2:{2}, 3:{4}, 4:{4}, 5:{6}, 6:{6}, 7:{8}, 8:{8}},
                 backward={1:{1}, 2:{1}, 3:{3}, 4:{3}, 5:{5}, 6:{5}, 7:{7}, 8:{7}})

In [ ]:
# Some grid routing problems

# The following can be used to create obstacles:
    
def random_lines(X=range(15, 130), Y=range(60), N=150, lengths=range(6, 12)):
    """The set of cells in N random lines of the given lengths."""
    result = set()
    for _ in range(N):
        x, y = random.choice(X), random.choice(Y)
        dx, dy = random.choice(((0, 1), (1, 0)))
        result |= line(x, y, dx, dy, random.choice(lengths))
    return result

def line(x, y, dx, dy, length):
    """A line of `length` cells starting at (x, y) and going in (dx, dy) direction."""
    return {(x + i * dx, y + i * dy) for i in range(length)}

random.seed(42) # To make this reproducible

frame = line(-10, 20, 0, 1, 20) | line(150, 20, 0, 1, 20)
cup = line(102, 44, -1, 0, 15) | line(102, 20, -1, 0, 20) | line(102, 44, 0, -1, 24)

d1 = GridProblem(obstacles=random_lines(N=100) | frame)
d2 = GridProblem(obstacles=random_lines(N=150) | frame)
d3 = GridProblem(obstacles=random_lines(N=200) | frame)
d4 = GridProblem(obstacles=random_lines(N=250) | frame)
d5 = GridProblem(obstacles=random_lines(N=300) | frame)
d6 = GridProblem(obstacles=cup | frame)
d7 = GridProblem(obstacles=cup | frame | line(50, 35, 0, -1, 10) | line(60, 37, 0, -1, 17) | line(70, 31, 0, -1, 19))

# 8 Puzzle Problems

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

A sliding tile puzzle where you can swap the blank with an adjacent piece, trying to reach a goal configuration. The cells are numbered 0 to 8, starting at the top left and going row by row left to right. The pieces are numebred 1 to 8, with 0 representing the blank. An action is the cell index number that is to be swapped with the blank (*not* the actual number to be swapped but the index into the state). So the diagram above left is the state `(5, 2, 7, 8, 4, 0, 1, 3, 6)`, and the action is `8`, because the cell number 8 (the 9th or last cell, the `6` in the bottom right) is swapped with the blank.

There are two disjoint sets of states that cannot be reached from each other. One set has an even number of "inversions"; the other has an odd number. An inversion is when a piece in the state is larger than a piece that follows it.




In [ ]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """
    def __init__(self, initial, goal=(1, 2, 3, 8, 0, 4, 7, 6, 5)):
    # def __init__(self, initial, goal=(0, 1, 2, 3, 4, 5, 6, 7, 8)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can be moved to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    # action cost other than default 1
    #def action_cost(self, s, a, s1): return math.exp(s[a]);
    #def action_cost(self, s, a, s1): return s[a]

    
    # heuristic functions
    def h0(self, node):
        """The null heuristic."""
        return 0

    def h1(self, node):
        """The misplaced tiles heuristic. It has a bug as it counts 0 as avalid tilde!!"""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Manhattan heuristic for goal (0, 1, 2, 3, 4, 5, 6, 7, 8)"""
        X = (0, 1, 2, 0, 1, 2, 0, 1, 2)
        Y = (0, 0, 0, 1, 1, 1, 2, 2, 2)
        return sum(abs(X[s] - X[g]) + abs(Y[s] - Y[g])
                   for (s, g) in zip(node.state, self.goal) if s != 0)
                   # g = 0, 1, ..., 8 (tuple positions, or tildes in goal.state)
                   # s = gth tilde in the node.state
                   # X[g],Y[g] = board position of tilde s in node.state
                   # X[s],Y[s] = board position of tilde s in goal.state = (0, 1, 2, 3, 4, 5, 6, 7, 8)

    def h2_generic(self, node):
        """Manhattan heuristic for any goal"""
        acc = 0
        for index in range(9):
            tile = node.state[index]

            if(tile == 0): continue # do not count tile 0

            goalpos = self.goal.index(tile)

            manhattan_x = abs(index % 3 - goalpos % 3)
            manhattan_y = abs(index // 3 - goalpos // 3)

            acc += manhattan_x + manhattan_y
        return acc

    def h3_generic(self, node): # Only sums when tile is distance == 2
        """Manhattan heuristic for any goal"""
        acc = 0
        for index in range(9):
            tile = node.state[index]

            if(tile == 0): continue # do not count tile 0

            goalpos = self.goal.index(tile)

            manhattan_x = abs(index % 3 - goalpos % 3)
            manhattan_y = abs(index // 3 - goalpos // 3)

            if(manhattan_x + manhattan_y == 2):
                acc += manhattan_x + manhattan_y
        return acc

    
    # Actual heuristic
    def h(self, node): 
       return self.h3_generic(node)
    
    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b and a != 0 for a, b in zip(A, B))
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')



In [ ]:
# Some specific EightPuzzle problems, reachable from (1, 2, 3, 8, 0, 4, 7, 6, 5)

# e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
# e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 1, 5, 3, 7, 8, 6))
# e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
# e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))
# e6 = EightPuzzle((8, 2, 0, 3, 4, 5, 6, 1, 7))
e7 = EightPuzzle((8, 0, 3, 2, 4, 5, 6, 1, 7))
e8 = EightPuzzle((6, 3, 2, 5, 8, 1, 4, 0, 7))

e30 = EightPuzzle((5, 6,7,2,8,4,0,3,1))

In [ ]:
# Solve an 8 puzzle problem and print out each state
for s in path_states(astar_search(e30)):
    print(board8(s))

# Water Pouring Problems

![](http://puzzles.nigelcoldwell.co.uk/images/water22.png)

In a [water pouring problem](https://en.wikipedia.org/wiki/Water_pouring_puzzle) you are given a collection of jugs, each of which has a size (capacity) in, say, litres, and a current level of water (in litres). The goal is to measure out a certain level of water; it can appear in any of the jugs. For example, in the movie *Die Hard 3*, the heroes were faced with the task of making exactly 4 gallons from jugs of size 5 gallons and 3 gallons.) A state is represented by a tuple of current water levels, and the available actions are:
- `(Fill, i)`: fill the `i`th jug all the way to the top (from a tap with unlimited water).
- `(Dump, i)`: dump all the water out of the `i`th jug.
- `(Pour, i, j)`: pour water from the `i`th jug into the `j`th jug until either the jug `i` is empty, or jug `j` is full, whichever comes first.

In [ ]:
class PourProblem(Problem):
    """Problem about pouring water between jugs to achieve some water level.
    Each state is a tuples of water levels. In the initialization, also provide a tuple of 
    jug sizes, e.g. PourProblem(initial=(0, 0), goal=4, sizes=(5, 3)), 
    which means two jugs of sizes 5 and 3, initially both empty, with the goal
    of getting a level of 4 in either jug."""
    
    def actions(self, state):
        """The actions executable in this state."""
        jugs = range(len(state))
        return ([('Fill', i)    for i in jugs if state[i] < self.sizes[i]] +
                [('Dump', i)    for i in jugs if state[i]] +
                [('Pour', i, j) for i in jugs if state[i] for j in jugs if i != j])

    def result(self, state, action):
        """The state that results from executing this action in this state."""
        result = list(state)
        act, i, *_ = action
        if act == 'Fill':   # Fill i to capacity
            result[i] = self.sizes[i]
        elif act == 'Dump': # Empty i
            result[i] = 0
        elif act == 'Pour': # Pour from i into j
            j = action[2]
            amount = min(state[i], self.sizes[j] - state[j])
            result[i] -= amount
            result[j] += amount
        return tuple(result)

    def is_goal(self, state):
        """True if the goal level is in any one of the jugs."""
        return self.goal in state

In a `GreenPourProblem`, the states and actions are the same, but instead of all actions costing 1, in these problems the cost of an action is the amount of water that flows from the tap. (There is an issue that non-*Fill* actions have 0 cost, which in general can lead to indefinitely long solutions, but in this problem there is a finite number of states, so we're ok.)

In [ ]:
class GreenPourProblem(PourProblem): 
    """A PourProblem in which the cost is the amount of water used."""
    def action_cost(self, s, action, s1):
        "The cost is the amount of water used."
        act, i, *_ = action
        return self.sizes[i] - s[i] if act == 'Fill' else 0

In [ ]:
# Some specific PourProblems

p1 = PourProblem((1, 1, 1), 13, sizes=(2, 16, 32))
p2 = PourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
p3 = PourProblem((0, 0),     8, sizes=(7,9))
p4 = PourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
p5 = PourProblem((0, 0),     4, sizes=(3, 5))

g1 = GreenPourProblem((1, 1, 1), 13, sizes=(2, 16, 32))
g2 = GreenPourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
g3 = GreenPourProblem((0, 0),     8, sizes=(7,9))
g4 = GreenPourProblem((0, 0, 0), 21, sizes=(8, 11, 31))
g5 = GreenPourProblem((0, 0),     4, sizes=(3, 5))

In [ ]:
# Solve the PourProblem of getting 13 in some jug, and show the actions and states
soln = breadth_first_search(p1)
path_actions(soln), path_states(soln)

# Pancake Sorting Problems

Given a stack of pancakes of various sizes, can you sort them into a stack of decreasing sizes, largest on bottom to smallest on top? You have a spatula with which you can flip the top `i` pancakes. This is shown below for `i = 3`; on the top the spatula grabs the first three pancakes; on the bottom we see them flipped:


![](https://upload.wikimedia.org/wikipedia/commons/0/0f/Pancake_sort_operation.png)

How many flips will it take to get the whole stack sorted? This is an interesting [problem](https://en.wikipedia.org/wiki/Pancake_sorting) that Bill Gates has [written about](https://people.eecs.berkeley.edu/~christos/papers/Bounds%20For%20Sorting%20By%20Prefix%20Reversal.pdf). A reasonable heuristic for this problem is the *gap heuristic*: if we look at neighboring pancakes, if, say, the 2nd smallest is next to the 3rd smallest, that's good; they should stay next to each other. But if the 2nd smallest is next to the 4th smallest, that's bad: we will require at least one move to separate them and insert the 3rd smallest between them. The gap heuristic counts the number of neighbors that have a gap like this. In our specification of the problem, pancakes are ranked by size: the smallest is `1`, the 2nd smallest `2`, and so on, and the representation of a state is a tuple of these rankings, from the top to the bottom pancake. Thus the goal state is always `(1, 2, ..., `*n*`)` and the initial (top) state in the diagram above is `(2, 1, 4, 6, 3, 5)`.


In [ ]:
class PancakeProblem(Problem):
    """A PancakeProblem the goal is always `tuple(range(1, n+1))`, where the
    initial state is a permutation of `range(1, n+1)`. An act is the index `i` 
    of the top `i` pancakes that will be flipped."""
    
    def __init__(self, initial): 
        self.initial, self.goal = tuple(initial), tuple(sorted(initial))
    
    def actions(self, state): return range(2, len(state) + 1)

    def result(self, state, i): return state[:i][::-1] + state[i:]
    
    def h(self, node):
        "The gap heuristic."
        s = node.state
        return sum(abs(s[i] - s[i - 1]) > 1 for i in range(1, len(s)))

In [ ]:
c0 = PancakeProblem((2, 1, 4, 6, 3, 5))
c1 = PancakeProblem((4, 6, 2, 5, 1, 3))
c2 = PancakeProblem((1, 3, 7, 5, 2, 6, 4))
c3 = PancakeProblem((1, 7, 2, 6, 3, 5, 4))
c4 = PancakeProblem((1, 3, 5, 7, 9, 2, 4, 6, 8))

In [ ]:
# Solve a pancake problem
path_states(astar_search(c0))

# Jumping Frogs Puzzle

In this puzzle (which also can be played as a two-player game), the initial state is a line of squares, with N pieces of one kind on the left, then one empty square, then N pieces of another kind on the right. The diagram below uses 2 blue toads and 2 red frogs; we will represent this as the string `'LL.RR'`. The goal is to swap the pieces, arriving at `'RR.LL'`. An `'L'` piece moves left-to-right, either sliding one space ahead to an empty space, or two spaces ahead if that space is empty and if there is an `'R'` in between to hop over. The `'R'` pieces move right-to-left analogously. An action will be an `(i, j)` pair meaning to swap the pieces at those indexes. The set of actions for the N = 2 position below is `{(1, 2), (3, 2)}`, meaning either the blue toad in position 1 or the red frog in position 3 can swap places with the blank in position 2.

![](https://upload.wikimedia.org/wikipedia/commons/2/2f/ToadsAndFrogs.png)

In [ ]:
class JumpingPuzzle(Problem):
    """Try to exchange L and R by moving one ahead or hopping two ahead."""
    def __init__(self, N=2):
        self.initial = N*'L' + '.' + N*'R'
        self.goal = self.initial[::-1]
        
    def actions(self, state):
        """Find all possible move or hop moves."""
        idxs = range(len(state))
        return ({(i, i + 1) for i in idxs if state[i:i+2] == 'L.'}   # Slide
               |{(i, i + 2) for i in idxs if state[i:i+3] == 'LR.'}  # Hop
               |{(i + 1, i) for i in idxs if state[i:i+2] == '.R'}   # Slide
               |{(i + 2, i) for i in idxs if state[i:i+3] == '.LR'}) # Hop

    def result(self, state, action):
        """An action (i, j) means swap the pieces at positions i and j."""
        i, j = action
        result = list(state)
        result[i], result[j] = state[j], state[i]
        return ''.join(result)
    
    def h(self, node): return hamming_distance(node.state, self.goal)

In [ ]:
JumpingPuzzle(N=2).actions('LL.RR')

In [ ]:
j3 = JumpingPuzzle(N=3)
j9 = JumpingPuzzle(N=9)
path_states(astar_search(j3))

# Reporting Summary Statistics on Search Algorithms

Now let's gather some metrics on how well each algorithm does.  We'll use `CountCalls` to wrap a `Problem` object in such a way that calls to its methods are delegated to the original problem, but each call increments a counter. Once we've solved the problem, we print out summary statistics.

In [ ]:
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))

Here's a tiny report for uniform-cost search on the jug pouring problems:

In [ ]:
report([uniform_cost_search], [p1, p2, p3, p4, p5])

In [ ]:
report((uniform_cost_search, breadth_first_search), 
       (p1, g1, p2, g2, p3, g3, p4, g4, p4, g4, c1, c2, c3)) 

# Comparing heuristics

First, let's look at the eight puzzle problems, and compare three different heuristics the Manhattan heuristic, the less informative misplaced tiles heuristic, and the uninformed (i.e. *h* = 0) breadth-first search:

In [ ]:
def astar_misplaced_tiles(problem): return astar_search(problem, h=problem.h1)

report([breadth_first_search, astar_misplaced_tiles, astar_search], 
       [e1, e2, e3, e4, e5])

We see that all three algorithms get cost-optimal solutions, but the better the heuristic, the fewer nodes explored. 
Compared to the uninformed search, the misplaced tiles heuristic explores about 1/4 the number of nodes, and the Manhattan heuristic needs just 2%.

Next, we can show the value of the gap heuristic for pancake sorting problems:

In [ ]:
report([astar_search, uniform_cost_search], [c1, c2, c3, c4])

We need to explore 300 times more nodes without the heuristic.

# Comparing graph search and tree search

Keeping the *reached* table in `best_first_search` allows us to do a graph search, where we notice when we reach a state by two different paths, rather than a tree search, where we have duplicated effort. The *reached* table consumes space and also saves time. How much time? In part it depends on how good the heuristics are at focusing the search.  Below we show that on some pancake and eight puzzle problems, the tree search expands roughly twice as many nodes (and thus takes roughly twice as much time):

In [ ]:
report([astar_search, astar_tree_search], [e1, e2, e3, e4, r1, r2, r3, r4])

# Comparing different weighted search values

Below we report on problems using these four algorithms:

|Algorithm|*f*|Optimality|
|:---------|---:|:----------:|
|Greedy best-first search | *f = h*|nonoptimal|
|Extra weighted A* search | *f = g + 2 &times; h*|nonoptimal|
|Weighted A* search | *f = g + 1.4 &times; h*|nonoptimal|
|A* search | *f = g + h*|optimal|
|Uniform-cost search | *f = g*|optimal|

We will see that greedy best-first search (which ranks nodes solely by the heuristic) explores the fewest number of nodes, but has the highest path costs. Weighted A* search explores twice as many nodes (on this problem set) but gets 10% better path costs. A* is optimal, but explores more nodes, and uniform-cost is also optimal, but explores an order of magnitude more nodes.

In [ ]:
def extra_weighted_astar_search(problem): return weighted_astar_search(problem, weight=2)
    
report((greedy_bfs, extra_weighted_astar_search, weighted_astar_search, astar_search, uniform_cost_search), 
       (r0, r1, r2, r3, r4, e1, d1, d2, j9, e2, d3, d4, d6, d7, e3, e4))

We see that greedy search expands the fewest nodes, but has the highest path costs. In contrast, A\* gets optimal path costs, but expands 4 or 5 times more nodes. Weighted A* is a good compromise, using half the compute time as A\*, and achieving path costs within  1% or 2% of optimal. Uniform-cost is optimal, but is an order of magnitude slower than A\*.

# Comparing  many search algorithms

Finally, we compare a host of algorihms (even the slow ones) on some of the easier problems:

In [ ]:
report((astar_search, uniform_cost_search,  breadth_first_search, breadth_first_bfs, 
        iterative_deepening_search, depth_limited_search, greedy_bfs, 
        weighted_astar_search, extra_weighted_astar_search), 
       (p1, g1, p2, g2, p3, g3, p4, g4, r0, r1, r2, r3, r4, e1))

This confirms some of the things we already knew: A* and uniform-cost search are optimal, but the others are not. A* explores fewer nodes than uniform-cost. 

# Visualizing Reached States

I would like to draw a picture of the state space, marking the states that have been reached by the search.
Unfortunately, the *reached* variable is inaccessible inside `best_first_search`, so I will define a new version of `best_first_search` that is identical except that it declares *reached* to be `global`. I can then define `plot_grid_problem` to plot the obstacles of a `GridProblem`, along with the initial and goal states, the solution path, and the states reached during a search.

In [ ]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    global reached # <<<<<<<<<<< Only change here
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def plot_grid_problem(grid, solution, reached=(), title='Search', show=True):
    "Use matplotlib to plot the grid, obstacles, solution, and reached."
    reached = list(reached)
    plt.figure(figsize=(16, 10))
    plt.axis('off'); plt.axis('equal')
    plt.scatter(*transpose(grid.obstacles), marker='s', color='darkgrey')
    plt.scatter(*transpose(reached), 1**2, marker='.', c='blue')
    plt.scatter(*transpose(path_states(solution)), marker='s', c='blue')
    plt.scatter(*transpose([grid.initial]), 9**2, marker='D', c='green')
    plt.scatter(*transpose([grid.goal]), 9**2, marker='8', c='red')
    if show: plt.show()
    print('{} {} search: {:.1f} path cost, {:,d} states reached'
          .format(' ' * 10, title, solution.path_cost, len(reached)))
    
def plots(grid, weights=(1.4, 2)): 
    """Plot the results of 4 heuristic search algorithms for this grid."""
    solution = astar_search(grid)
    plot_grid_problem(grid, solution, reached, 'A* search')
    for weight in weights:
        solution = weighted_astar_search(grid, weight=weight)
        plot_grid_problem(grid, solution, reached, '(b) Weighted ({}) A* search'.format(weight))
    solution = greedy_bfs(grid)
    plot_grid_problem(grid, solution, reached, 'Greedy best-first search')
    
def transpose(matrix): return list(zip(*matrix))

In [ ]:
plots(d3)

In [ ]:
plots(d4)

# The cost of weighted A* search

Now I want to try a much simpler grid problem, `d6`, with only a few obstacles. We see that A* finds the optimal path, skirting below the obstacles. Weighterd A* with a weight of 1.4 finds the same optimal path while exploring only 1/3 the number of states. But weighted A* with weight 2 takes the slightly longer path above the obstacles, because that path allowed it to stay closer to the goal in straight-line distance, which it over-weights. And greedy best-first search has a bad showing, not deviating from its path towards the goal until it is almost inside the cup made by the obstacles.

In [ ]:
plots(d6)

In the next problem, `d7`, we see a similar story. the optimal path found by A*, and we see that again weighted A* with weight 1.4 does great and with weight 2 ends up erroneously going below the first two barriers, and then makes another mistake by reversing direction back towards the goal and passing above the third barrier. Again, greedy best-first makes bad decisions all around.

In [ ]:
plots(d7)

# Nondeterministic Actions

To handle problems with nondeterministic problems, we'll replace the `result` method with `results`, which returns a collection of possible result states. We'll represent the solution to a problem not with a `Node`, but with a plan that consist of two types of component: sequences of actions, like `['forward', 'suck']`, and condition actions, like
`{5: ['forward', 'suck'], 7: []}`, which says that if we end up in state 5, then do `['forward', 'suck']`, but if we end up in state 7, then do the empty sequence of actions.

In [ ]:
def and_or_search(problem):
    "Find a plan for a problem that has nondterministic actions."
    return or_search(problem, problem.initial, [])
    
def or_search(problem, state, path):
    "Find a sequence of actions to reach goal from state, without repeating states on path."
    if problem.is_goal(state): return []
    if state in path: return failure # check for loops
    for action in problem.actions(state):
        plan = and_search(problem, problem.results(state, action), [state] + path)
        if plan != failure:
            return [action] + plan
    return failure

def and_search(problem, states, path):
    "Plan for each of the possible states we might end up in."
    if len(states) == 1: 
        return or_search(problem, next(iter(states)), path)
    plan = {}
    for s in states:
        plan[s] = or_search(problem, s, path)
        if plan[s] == failure: return failure
    return [plan]

In [ ]:
class MultiGoalProblem(Problem):
    """A version of `Problem` with a colllection of `goals` instead of one `goal`."""
    
    def __init__(self, initial=None, goals=(), **kwds): 
        self.__dict__.update(initial=initial, goals=goals, **kwds)
        
    def is_goal(self, state): return state in self.goals
    
class ErraticVacuum(MultiGoalProblem):
    """In this 2-location vacuum problem, the suck action in a dirty square will either clean up that square,
    or clean up both squares. A suck action in a clean square will either do nothing, or
    will deposit dirt in that square. Forward and backward actions are deterministic."""
    
    def actions(self, state): 
        return ['suck', 'forward', 'backward']
    
    def results(self, state, action): return self.table[action][state]
    
    table = {'suck':{1:{5,7}, 2:{4,8}, 3:{7}, 4:{2,4}, 5:{1,5}, 6:{8}, 7:{3,7}, 8:{6,8}},
             'forward': {1:{2}, 2:{2}, 3:{4}, 4:{4}, 5:{6}, 6:{6}, 7:{8}, 8:{8}},
             'backward': {1:{1}, 2:{1}, 3:{3}, 4:{3}, 5:{5}, 6:{5}, 7:{7}, 8:{7}}}

Let's find a plan to get from state 1 to the goal of no dirt (states 7 or 8):

In [ ]:
and_or_search(ErraticVacuum(1, {7, 8}))

This plan says "First suck, and if we end up in state 5, go forward and suck again; if we end up in state 7, do nothing because that is a goal."

Here are the plans to get to a goal state starting from any one of the 8 states:

In [ ]:
{s: and_or_search(ErraticVacuum(s, {7,8})) 
 for s in range(1, 9)}

# Comparing Algorithms on EightPuzzle Problems of Different Lengths

In [ ]:
from functools import lru_cache

def build_table(table, depth, state, problem):
    if depth > 0 and state not in table:
        problem.initial = state
        table[state] = len(astar_search(problem))
        for a in problem.actions(state):
            build_table(table, depth - 1, problem.result(state, a), problem)
    return table

def invert_table(table):
    result = defaultdict(list)
    for key, val in table.items():
        result[val].append(key)
    return result

goal = (0, 1, 2, 3, 4, 5, 6, 7, 8)
table8 = invert_table(build_table({}, 25, goal, EightPuzzle(goal)))

In [ ]:
def report8(table8, M, Ds=range(2, 25, 2), searchers=(breadth_first_search, astar_misplaced_tiles, astar_search)):
    "Make a table of average nodes generated and effective branching factor"
    for d in Ds:
        line = [d]
        N = min(M, len(table8[d]))
        states = random.sample(table8[d], N)
        for searcher in searchers:
            nodes = 0
            for s in states:
                problem = CountCalls(EightPuzzle(s))
                searcher(problem)
                nodes += problem._counts['result']
            nodes = int(round(nodes/N))
            line.append(nodes)
        line.extend([ebf(d, n) for n in line[1:]])
        print('{:2} & {:6} & {:5} & {:5} && {:.2f} & {:.2f} & {:.2f}'
              .format(*line))

        
def ebf(d, N, possible_bs=[b/100 for b in range(100, 300)]):
    "Effective Branching Factor"
    return min(possible_bs, key=lambda b: abs(N - sum(b**i for i in range(1, d+1))))

def edepth_reduction(d, N, b=2.67):
    
    

from statistics import mean 

def random_state():
    x = list(range(9))
    random.shuffle(x)
    return tuple(x)

meanbf = mean(len(e3.actions(random_state())) for _ in range(10000))
meanbf

In [ ]:
{n: len(v) for (n, v) in table30.items()}

In [ ]:
%time table30 = invert_table(build_table({}, 30, goal, EightPuzzle(goal)))

In [ ]:
%time report8(table30, 20, range(26, 31, 2))

In [ ]:
%time report8(table30, 20, range(26, 31, 2))

In [ ]:
from itertools import combinations
from statistics import median, mean

# Detour index for Romania

L = romania.locations
def ratio(a, b): return astar_search(RouteProblem(a, b, map=romania)).path_cost / sld(L[a], L[b])
nums = [ratio(a, b) for a,b in combinations(L, 2) if b in r1.actions(a)]
mean(nums), median(nums) # 1.7, 1.6 # 1.26, 1.2 for adjacent cities

In [ ]:
sld